# Implementação de Word2Vec
Segue abaixo um exemplo trivial de Word2Vec na prática. Vale observar que o modelo não tão preciso por causa da carência de dados de treino. Mas como o objetivo é apenas fazer uma demonstração, isso não será relevante.

In [ ]:
# ------------------------------------
# Baixando e importando as dependencias
# ------------------------------------
!pip install gensim nltk

import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')


In [28]:
# --------------------------------------
# Dados de Treinamento
# --------------------------------------
sentences = [
    # Intenção: redefinir_senha
    "como redefinir minha senha",
    "esqueci minha senha preciso de ajuda",
    "quero alterar minha senha",
    "preciso mudar minha senha",
    "como trocar a senha do meu acesso",

    # Intenção: rastrear_pedido
    "como rastrear meu pedido",
    "onde está meu pacote",
    "status do meu pedido",
    "quero saber onde está minha encomenda",
    "como acompanhar a entrega do meu produto",

    # Intenção: cancelar_conta
    "quero cancelar minha assinatura",
    "como encerrar minha conta",
    "desejo deletar meu perfil",
    "como posso excluir minha conta",
    "preciso encerrar meu cadastro"
]


In [29]:
# --------------------------------------
# Pré-processamento
# --------------------------------------
stopwords = nltk.corpus.stopwords.words('portuguese')
additional_stopwords = ["como", "quero", "preciso", "desejo"]  # Palavras específicas do domínio
stopwords += additional_stopwords

def preprocess(text):
    tokens = nltk.word_tokenize(text.lower(), language='portuguese')
    tokens = [word for word in tokens if word.isalpha() and word not in stopwords]
    return tokens

processed_sentences = [preprocess(sentence) for sentence in sentences]


In [30]:
# --------------------------------------
# Treinamento do Modelo com Parâmetros Ajustados
# --------------------------------------
model = Word2Vec(
    sentences=processed_sentences,
    vector_size=150,  # Aumentado para capturar mais nuances
    window=7,         # Janela maior para contexto amplo
    min_count=1,
    workers=4,
    epochs=50         # Mais épocas de treinamento
)


In [31]:
# --------------------------------------
# Mapeamento de Intenções com Palavras-Chave Aprimoradas
# --------------------------------------
intent_vectors = {
    "redefinir_senha": np.mean([
        model.wv["redefinir"],
        model.wv["senha"],
        model.wv["alterar"],
        model.wv["mudar"]
    ], axis=0),

    "rastrear_pedido": np.mean([
        model.wv["rastrear"],
        model.wv["pedido"],
        model.wv["pacote"],
        model.wv["encomenda"],
        model.wv["entrega"]
    ], axis=0),

    "cancelar_conta": np.mean([
        model.wv["cancelar"],
        model.wv["conta"],
        model.wv["excluir"],
        model.wv["encerrar"]
    ], axis=0)
}

In [32]:
# --------------------------------------
# Função de Predição com Limiar de Confiança
# --------------------------------------
def predict_intent(query):
    tokens = preprocess(query)
    if not tokens:
        return "Intenção não reconhecida"

    # Calcula vetor apenas com palavras conhecidas
    valid_vectors = [model.wv[word] for word in tokens if word in model.wv]
    if not valid_vectors:
        return "Intenção não reconhecida"

    input_vector = np.mean(valid_vectors, axis=0)

    best_intent = None
    best_similarity = -1

    for intent, vec in intent_vectors.items():
        similarity = cosine_similarity([input_vector], [vec])[0][0]
        if similarity > best_similarity:
            best_similarity = similarity
            best_intent = intent

    # Limiar de confiança (ajustável conforme necessidade)
    if best_similarity < 0.4:  # Rejeita similaridades baixas
        return "Intenção não reconhecida"

    return best_intent

In [33]:
# --------------------------------------
# Testes Atualizados
# --------------------------------------
test_queries = [
    "mudar minha senha",
    "onde está minha encomenda",
    "apagar conta",
    "parar assinatura",
    "como acompanho meu pacote",
    "quero excluir meu cadastro"
]

for query in test_queries:
    print(f"'{query}' → {predict_intent(query)}")

'mudar minha senha' → redefinir_senha
'onde está minha encomenda' → rastrear_pedido
'apagar conta' → cancelar_conta
'parar assinatura' → Intenção não reconhecida
'como acompanho meu pacote' → rastrear_pedido
'quero excluir meu cadastro' → Intenção não reconhecida


# Exemplo de implementação de RNN(LSTM)
O objetivo simular que o Chatbot agora vai aplicar os conceitos vistos nessa oficina.

In [37]:
!pip install tensorflow gensim numpy scikit-learn --quiet


In [43]:
# 2. Importar as bibliotecas e preparar os dados

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from tensorflow.keras.utils import to_categorical

# Exemplo de dados em português
texts = [
    "Eu adoro este produto",
    "Não gostei, achei caro e já fiu",
    "Estou muito feliz com o meu pedido",
    "Isso é um absurdo, que decepção"
]

labels = ["positivo", "negativo", "positivo", "negativo"]

# Convertendo textos para listas de palavras
processed_texts = [simple_preprocess(text) for text in texts]

In [44]:
# 3. Treinar o modelo Word2Vec

word2vec_model = Word2Vec(sentences=processed_texts, vector_size=100, window=5, min_count=1, sg=0)
word2vec_model.wv.save("word2vec.wordvectors")
vocabulary = word2vec_model.wv.key_to_index

In [45]:
# 4. Preparar os dados para o modelo LSTM

# Mapear palavras para índices
def texts_to_sequences(texts, word2vec_model):
    sequences = []
    for text in texts:
        seq = [word2vec_model.wv.key_to_index[word] for word in text if word in word2vec_model.wv]
        sequences.append(seq)
    return sequences

sequences = texts_to_sequences(processed_texts, word2vec_model)
max_len = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Codificar rótulos
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)
one_hot_labels = to_categorical(encoded_labels)

In [46]:
# 5. Dividir o conjunto de dados e construir o modelo LSTM

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_labels, test_size=0.2, random_state=42)

# Construir o modelo LSTM
model = Sequential()
model.add(Embedding(input_dim=len(vocabulary), output_dim=100, weights=[word2vec_model.wv.vectors], input_length=max_len, trainable=False))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(2, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Treino do modelo
model.fit(X_train, y_train, epochs=5, batch_size=2, validation_split=0.1)

# Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print("Acurácia:", accuracy)
print("Função perca", loss)

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 0.6938 - val_accuracy: 1.0000 - val_loss: 0.6928
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - accuracy: 0.5000 - loss: 0.6921 - val_accuracy: 0.0000e+00 - val_loss: 0.6954
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 1.0000 - loss: 0.6904 - val_accuracy: 0.0000e+00 - val_loss: 0.6959
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 1.0000 - loss: 0.6887 - val_accuracy: 0.0000e+00 - val_loss: 0.6951
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 1.0000 - loss: 0.6869 - val_accuracy: 0.0000e+00 - val_loss: 0.6948
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 1.0000 - loss: 0.6927
Acurácia: 1.0
Função perca 0.6926566958427429
